In [12]:
import moviepy.editor as mp
from pydub import AudioSegment
from pydub.silence import split_on_silence
from vosk import Model, KaldiRecognizer
import subprocess
from pathlib import Path
import numpy as np

# Set log level to suppress Vosk log messages
SetLogLevel(0)

# Step 1: Extract audio from video
video = mp.VideoFileClip("um.mp4")
audio = video.audio


# Step 2: Convert audio to WAV for speech recognition
audio.write_audiofile("audio.wav", codec="pcm_s16le")

# Step 3: Speech recognition using Vosk (you'll need to install the vosk library)
model_path = "../vosk-model-en-us-0.22"
try:
    model = Model(model_path)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    # Handle the error or exit the program


if 'model' in locals():
    # Read back the audio as a byte buffer
    with open("audio.wav", "rb") as f:
        audio_bytes = f.read()

    # Convert audio data to list of 16-bit signed integers
    audio_list = list(np.frombuffer(audio_bytes, dtype=np.int16))
    print(audio_list)

    recognizer = KaldiRecognizer(model, audio_list)
    recognizer.SetWords(True)
    
    # Recognize speech
    recognized_text = ""
    while True:
        data = recognizer.ReadWaveChunk()
        if not data:
            break
        recognized_text += recognizer.PartialResult()
    # Rest of the code...
else:
    print("Model not available, cannot proceed.")
# Rest of the code...

audio_data = AudioSegment.from_wav("audio.wav")
chunks = split_on_silence(audio_data, silence_thresh=-40)
recognized_text = ""

for chunk in chunks:
     # Convert audio data to numpy array of 16-bit signed integers
    audio_np = np.frombuffer(audio_bytes, dtype=np.int16)
    recognizer = KaldiRecognizer(model, audio_np)
    recognized_text += recognizer.FinalResult()

# Step 4: Identify "um" occurrences and cut corresponding video segments
um_segments = [(start, end) for start, end in zip(chunks[:-1], chunks[1:]) if "um" in recognized_text[start:end]]

edited_segments = []
for start, end in um_segments:
    edited_segments.append(video.subclip(start, end))

final_video = mp.concatenate_videoclips(edited_segments)

# Step 5: Export final video
final_video.write_videofile("umRemove.mp4")


Exception ignored in: <function KaldiRecognizer.__del__ at 0x000001A185B06E50>
Traceback (most recent call last):
  File "C:\Users\Name\anaconda3\envs\newdev\lib\site-packages\vosk\__init__.py", line 161, in __del__
AttributeError: 'KaldiRecognizer' object has no attribute '_handle'


MoviePy - Writing audio in audio.wav


MoviePy - Done.
Model loaded successfully.


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



TypeError: must be real number, not list